In [1]:
!gdown 1G7u9RBG_2_vY889Fsb2hMGz_AB8jzJMR
!mkdir dataset
!tar -xvf  'imagenet-a.tar' -C 'dataset'

/bin/bash: line 1: gdown: command not found
mkdir: cannot create directory ‘dataset’: File exists
tar: imagenet-a.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [2]:
!pip install -q ftfy regex tqdm scikit-learn scikit-image
!pip install -q git+https://github.com/openai/CLIP.git


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision.transforms.v2 as transforms
from PIL import Image
from pkg_resources import packaging
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

from clip import clip

%matplotlib inline
%config InlineBackend.figure_format = "retina"

# Dataset

In [4]:
class DatasetImageNetA(Dataset):
    def __init__(self, dataset_path, transform=None, target_transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.target_transform = target_transform

        # Read the mapping file
        with open(os.path.join(dataset_path ,"README.txt"), "r") as f:
            lines = f.readlines()[12:] #skip first 12 lines

        # Create the mapping dictionary
        self.mapping = {}
        for line in lines:
            split_line = line.split()
            if len(split_line) > 1:
                numeric_id = split_line[0][1:]
                name = " ".join(split_line[1:]).strip()
                self.mapping[int(numeric_id)] = name.lower()

        # Create the labels list
        labels = []
        for cl in self.mapping.keys():
            for file_name in os.listdir(os.path.join(dataset_path, f"n{str(cl).zfill(8)}")):
                labels.append((cl, file_name))
        self.labels=labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.dataset_path, f"n{str(self.labels[idx][0]).zfill(8)}", self.labels[idx][1])
        image = Image.open(img_path)
        label = self.labels[idx][0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

# Dataloader

In [5]:
dataset = DatasetImageNetA("dataset/imagenet-a", transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
]))

train_loader = torch.utils.data.DataLoader(dataset, shuffle=True)


/home/ettore/.conda/envs/nlu/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


# Embedding labels in templates

In [6]:
# def embed_dataset_classnames(classnames):
#     # Create the list of descriptions and tokenize them
#     descriptions = [f"a photo of a {label}." for label in classnames]
#     text_tokens = clip.tokenize(descriptions).cuda()
    
#     # Get the normalized textual features
#     with torch.no_grad():
#         texts_z = clip.encode_text(text_tokens).float()
#         texts_z /= texts_z.norm(dim=-1, keepdim=True)
    
#     return classnames, descriptions, texts_z

def embed_dataset_classnames(classnames, model, templates=["a photo of a {}."]):

    texts_z_views = []
    for template in templates:
        descriptions = [template.format(c) for c in classnames]
        text_tokens = clip.tokenize(descriptions).cuda()
        
        # Get the normalized textual features
        with torch.no_grad():
            texts_z = model.encode_text(text_tokens).float()
            texts_z /= texts_z.norm(dim=-1, keepdim=True)
            texts_z_views.append(texts_z)

    # Evaluate the mean representation
    texts_z = torch.stack(texts_z_views).mean(dim=0)

    # Renormalise
    texts_z /= texts_z.norm(dim=-1, keepdim=True)
    
    return classnames, texts_z

# Test

In [7]:
def test_step_zero_shot_clip(net, data_loader, texts_z, device='cuda'):
    samples = 0.0
    cumulative_accuracy = 0.0

    # Set the network to evaluation mode
    net.eval()

    with torch.no_grad():
        # Iterate over the test set
        try:
            for batch_idx, (inputs, targets) in tqdm(enumerate(data_loader), total=len(data_loader), position=0, leave=True):
                # Load data into GPU
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                # Forward pass
                images_z = net.encode_image(inputs).float()
                outputs = (100 * images_z @ texts_z.T).softmax(dim=-1)

                # Fetch prediction and loss value
                samples += inputs.shape[0]
                _, predicted = outputs.max(1)

                # Compute accuracy
                cumulative_accuracy += predicted.eq(targets).sum().item()
        except KeyboardInterrupt:
            pass

    return cumulative_accuracy / samples * 100

In [8]:
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break
    

torch.Size([1, 3, 224, 224]) torch.Size([1])


In [9]:
model, preprocess = clip.load("RN50")
model = model.cuda().eval()

In [10]:
texts, texts_z = embed_dataset_classnames(dataset.mapping.values(), model)
test_accuracy = test_step_zero_shot_clip(model, train_loader , texts_z)


  5%|▍         | 358/7500 [00:05<01:43, 69.27it/s]


RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 224, 224] to have 3 channels, but got 1 channels instead